In [3]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import json
from bs4 import BeautifulSoup
import requests
import json
import re
import time
import string
import urllib
from selenium import webdriver
from lxml import etree
from fake_useragent import UserAgent

In [4]:
#camouflage
ua = UserAgent()
header = {'User-Agent':ua.random}


In [19]:

def get_ip():
    ip_list = []
    #Route
    url = 'https://www.xicidaili.com/nt/' #ip is time-effective, only crawling the first page
    #request
    response = requests.get(url=url,headers=header)
    #Set encoding
    response.encoding = response.apparent_encoding
    response = response.text

    response = etree.HTML(response)

    tr_list = response.xpath('//tr[@class="odd"]')
    for i in tr_list:
        #ip
        ip = i.xpath('./td[2]/text()')[0]
        #Port number
        port = i.xpath('./td[3]/text()')[0]
        #Agreement
        agreement = i.xpath('./td[6]/text()')[0]
        agreement = agreement.lower()
        #Complete assembly path
        ip = agreement + '://' + ip + ':' + port
        ip_list.append(ip)
    return ip_list


In [20]:
ip_list = get_ip()
print(ip_list[0])

http://114.55.95.112:8999


check whether the anti-clawer solution has worked

In [21]:
# define the url to scrapy
url_main='https://cosmily.com/micellar-and-cleansing-water'

for ip in ip_list:
    print("scrapy: ", url_main)
    # set the proxy
    proxy = urllib.request.ProxyHandler({'https':ip})
    # create an opener
    # opener = urllib.request.build_opener(proxy, urllib.request.HTTPHandler)
    opener = urllib.request.build_opener(proxy)
    # set opener as global variable
    urllib.request.install_opener(opener)
    try:
        response = requests.get(url_main, headers=header)
        break
    except:
        time.sleep(1)
        continue

html = BeautifulSoup(response.text, 'html.parser')


scrapy:  https://cosmily.com/micellar-and-cleansing-water


In [ ]:
driver = webdriver.Firefox(executable_path=r'D:\Part-Time\geckodriver-v0.26.0-win64\geckodriver.exe')
driver.get(url_main)

# define the three json parts to save the information
others_json = {}
harmfulness_json = {}
beneficial_json = {}

# report_json = {}
# component_json = {}
# component_ratio_json = {}
# component_json = {}

# define the list to save all product
products_inf=[]
harmfulness_title = []
harmfulness_ingredient_icon = []
harmfulness_undertitle = []
harmfulness_style = []
beneficial_ingredient_icon = [] 
beneficial_title = []
beneficial_undertitle = [] 
beneficial_style = []

# define the list to save products urls
search = []
# find the hrefs for products in main page
products_list = driver.find_elements_by_xpath("//a[starts-with(@class, 'mb-2') and contains(@class, 'product-filter-title')]")
# save all the products hrefs at one page
for product in products_list:
    search.append(product.get_attribute("href"))
# build a blank json file
with open('products.txt', 'w') as outfile:
    json.dump([], outfile)

# go through the products hrefs to grasp the information   
for url in search:
    driver.get(url)
    try:
        # grap title data
        category_names = driver.find_elements_by_xpath("//span[@itemprop='name']")
        others_json['category_first_name'] = category_names[2].text
        others_json['category_second_name'] = category_names[3].text
        # print(category_names[3].text)
        others_json['category_third_name'] = category_names[-1].text

        # grap product_images
        images_parent = driver.find_element_by_xpath("//div[@class='product-image']")
        others_json['product_images'] = images_parent.find_element_by_xpath(".//a/img").get_attribute("src")    

        # grap product_name
        product_name = driver.find_element_by_xpath("//h1[starts-with(@class, 'mt-0') and contains(@class, 'mb-2')]").text
        others_json['product_name'] = product_name
        
        try:
            # grap country_name 
            others_json['country_name'] = driver.find_element_by_xpath("//img[@class='ml-1']").get_attribute("data-original-title")

            # grap brand_name
            brand_name = driver.find_element_by_xpath("//div[@class='mb-2']")
            others_json['brand_name'] = brand_name.find_element_by_xpath(".//a").text
        except:
            others_json['country_name'] = []
            others_json['brand_name'] = []
            pass
        # grap description
        others_json['description'] = driver.find_element_by_xpath("//p[starts-with(@class,'mb-4') and contains(@class, 'px-3')]").text   


        # grap harmfulness_json
        two_categories = driver.find_elements_by_xpath("//div[@class = 'px-3']") 

        harmfulnesses = two_categories[0].find_elements_by_xpath(".//div[starts-with(@class, 'col-') and contains(@class, 'ingredient-analysis')]")
        
        for i in range(len(harmfulnesses)):
            harmfulness_ingredient_icon.append(harmfulnesses[i].find_element_by_xpath(".//div/div/div/img[@class='ingredient-icon']").get_attribute("src"))
            harmfulness_title.append(harmfulnesses[i].find_element_by_xpath(".//div/div/div/img[@class='ingredient-icon']").get_attribute("alt")) 
            harmfulness_undertitle.append(harmfulnesses[i].find_element_by_xpath(".//div/div/div/div/span[@class='undertitle']").text)
            harmfulness_style.append(harmfulnesses[i].find_element_by_xpath(".//div/div[starts-with(@class, 'col-2')]/b").text)
            # harmfulness_json["down"]
        harmfulness_json['ingredient-icon'] = harmfulness_title
        harmfulness_json['title'] = harmfulness_title
        harmfulness_json['undertitle'] = harmfulness_undertitle
        harmfulness_json['style'] = harmfulness_style
        # grap beneficial_json
        beneficial = two_categories[1].find_elements_by_xpath(".//div[starts-with(@class, 'col-') and contains(@class, 'ingredient-analysis')]")
        
        for i in range(len(beneficial)):
            beneficial_ingredient_icon.append(beneficial[i].find_element_by_xpath(".//div/div/div/img[@class='ingredient-icon']").get_attribute("src"))
            beneficial_title.append(beneficial[i].find_element_by_xpath(".//div/div/div/img[@class='ingredient-icon']").get_attribute("alt")) 
            beneficial_undertitle.append(beneficial[i].find_element_by_xpath(".//div/div/div/div/span[@class='undertitle']").text)
            beneficial_style.append(beneficial[i].find_element_by_xpath(".//div/div[starts-with(@class, 'col-2')]/b").text)
            # harmfulness_json["down"]
        beneficial_json["ingredient-icon"] = beneficial_ingredient_icon
        beneficial_json["title"] = beneficial_title
        beneficial_json["undertitle"] = beneficial_undertitle
        beneficial_json["style"] = beneficial_style
        # save all the information for one product
        # print(beneficial_json)
    except Exception as e:
        print(e)
        pass
        
    products_inf.append(others_json)
    products_inf.append(harmfulness_json)
    products_inf.append(beneficial_json)
# save the result to products.txt
with open('products.txt', mode='w', encoding='utf-8') as feedsjson:
    json.dump(products_inf, feedsjson)
